# Prueba de Milvus

Ejemplo de uso de Milvus siguiendo pasos del sitio oficial.

**Referencias**

- https://milvus.io/docs/example_code.md

In [1]:
#!pip3 install protobuf==3.20.0
#!pip3 install grpcio-tools
#!pip install pymilvus

In [2]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [3]:
connections.connect("default", host="localhost", port="19530")

Crear una colección.

In [4]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
hello_milvus = Collection("hello_milvus", schema)

Insertar vectores en una colección.

In [5]:
import random
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]

insert_result = hello_milvus.insert(entities)

Indexar embeddings.

In [6]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)


Status(code=0, message='')

Cargar colección en memoria para hacer búsqueda por similaridad (cómo escala para muchas imágenes?)

In [7]:
hello_milvus.load()

In [8]:
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])
for r in result:
    print(r)

['(distance: 0.0, id: 2998)', '(distance: 0.09056773781776428, id: 211)', '(distance: 0.11295150220394135, id: 2087)']
['(distance: 0.0, id: 2999)', '(distance: 0.17350970208644867, id: 734)', '(distance: 0.17697687447071075, id: 788)']


Query por criterio "clásico".

In [9]:
result = hello_milvus.query(expr="random > -14", output_fields=["random", "embeddings"])
#result

Query híbrida. Se agrega al anterior criterios de búsqueda por similaridad del ejemplo previo.

In [10]:
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > -12", output_fields=["random"])
result

Borrar la colección.

In [11]:
utility.drop_collection("hello_milvus")